This script will take the SIP model output and aggregate it to the county and tract boundaries

***the SIP output does not conform to CF conventions.  So you need to add the Lat and lon variables as dimensions to the tracer


STEP 1 - add latitude and longitude as dimensions to SIP output 

**this will add lat and lon as dimensions instead of row and col

In [6]:
import xarray as xr
import numpy as np
from pathlib import Path
from scipy.interpolate import griddata  # used only when lat/lon are 2-D

#
#CHANGE this based on the SIP netcdf you want to work with
#
nc_in = r"/glade/derecho/scratch/lacey/ForJenn/camxv72_cb6r5_2016_PM25_avg.nc"  
nc_out = r"/glade/derecho/scratch/boehnert/AQE/SIP/sept_camxv72_sip_pm25_avg.nc"


# --- CONFIG ---
#CHANGE the variable of interest
varname = "PM25"          # your data variable
latname = "latitude"      # variable containing latitude values
lonname = "longitude"     # variable containing longitude values

# Open
ds = xr.open_dataset(nc_in)

if varname not in ds:
    raise ValueError(f"{varname} not found in {nc_in}")

if latname not in ds or lonname not in ds:
    raise ValueError(f"Expected variables {latname} and {lonname} in {nc_in}")

da = ds[varname]
latv = ds[latname]
lonv = ds[lonname]

# Identify the row/col dims used by FPRM (assume the last two are horizontal)
if da.ndim < 2:
    raise ValueError(f"{varname} must be at least 2D (row, col). Got {da.dims}")

row_dim, col_dim = da.dims[-2], da.dims[-1]

def write_out(dataset, path):
    # Light compression and keep float32
    encoding = {v: {"zlib": True, "complevel": 4} for v in dataset.data_vars}
    # Preserve dtype for the main var
    encoding[varname] = {**encoding.get(varname, {}), "dtype": "float32", "_FillValue": np.float32(np.nan)}
    dataset.to_netcdf(path, encoding=encoding)

# ---------- CASE A: 1-D lat/lon (easy swap) ----------
if latv.ndim == 1 and lonv.ndim == 1 and latv.dims[0] == row_dim and lonv.dims[0] == col_dim:
    # Attach 1D coords and swap dims
    out = ds.assign_coords({
        "latitude": (row_dim, latv.values.astype(np.float32)),
        "longitude": (col_dim, lonv.values.astype(np.float32)),
    }).swap_dims({row_dim: "latitude", col_dim: "longitude"})

    # Make sure the main var uses float32
    out[varname] = out[varname].astype("float32")

    # CF-ish attrs
    out["latitude"].attrs.update(dict(standard_name="latitude", units="degrees_north"))
    out["longitude"].attrs.update(dict(standard_name="longitude", units="degrees_east"))

    write_out(out, nc_out)
    print(f"Wrote {nc_out} with {varname}(latitude, longitude).")

# ---------- CASE B: 2-D lat/lon (curvilinear → regrid) ----------
elif latv.ndim == 2 and lonv.ndim == 2 and latv.dims == (row_dim, col_dim) and lonv.dims == (row_dim, col_dim):

    # Build a regular 1-D lat/lon grid based on median spacing
    lat2d = np.asarray(latv.values, dtype=np.float64)
    lon2d = np.asarray(lonv.values, dtype=np.float64)

    # Estimate grid spacing (robust to NaNs)
    def med_step(a, axis):
        d = np.nanmedian(np.abs(np.diff(a, axis=axis)))
        return float(d) if np.isfinite(d) and d > 0 else None

    dlat = med_step(lat2d, axis=0) or med_step(lat2d, axis=1) or 0.01
    dlon = med_step(lon2d, axis=1) or med_step(lon2d, axis=0) or 0.01

    lat_min, lat_max = np.nanmin(lat2d), np.nanmax(lat2d)
    lon_min, lon_max = np.nanmin(lon2d), np.nanmax(lon2d)

    lat_new = np.arange(lat_min, lat_max + dlat * 0.5, dlat, dtype=np.float32)
    lon_new = np.arange(lon_min, lon_max + dlon * 0.5, dlon, dtype=np.float32)
    Lon_new, Lat_new = np.meshgrid(lon_new, lat_new)

    # Prepare input points for interpolation
    pts = np.column_stack((lon2d.ravel(), lat2d.ravel()))

    # Move all non-horizontal dims to a single leading axis for looping
    other_dims = da.dims[:-2]
    other_sizes = [da.sizes[d] for d in other_dims]
    flat_count = int(np.prod(other_sizes)) if other_sizes else 1

    data = np.asarray(da.values, dtype=np.float32)
    data = data.reshape((flat_count, data.shape[-2], data.shape[-1]))

    out_stack = np.full((flat_count, lat_new.size, lon_new.size), np.nan, dtype=np.float32)

    for i in range(flat_count):
        vals = data[i].ravel()
        mask = np.isfinite(vals)
        if mask.sum() == 0:
            continue
        # 'nearest' is safest for coverage; change to 'linear' if desired
        interp = griddata(pts[mask], vals[mask],
                          (Lon_new, Lat_new),
                          method="nearest", fill_value=np.nan)
        out_stack[i, :, :] = interp.astype(np.float32)

    # Reshape back and build DataArray
    if other_dims:
        out_data = out_stack.reshape(*other_sizes, lat_new.size, lon_new.size)
        out_dims = [*other_dims, "latitude", "longitude"]
        coords = {d: ds.coords[d] if d in ds.coords else np.arange(ds.sizes[d]) for d in other_dims}
    else:
        out_data = out_stack[0]
        out_dims = ["latitude", "longitude"]
        coords = {}

    coords.update({
        "latitude": ("latitude", lat_new, {"standard_name":"latitude","units":"degrees_north"}),
        "longitude": ("longitude", lon_new, {"standard_name":"longitude","units":"degrees_east"}),
    })

    out_da = xr.DataArray(out_data, dims=out_dims, coords=coords, name=varname, attrs=da.attrs)
    out = xr.Dataset({varname: out_da})

    write_out(out, nc_out)
    print(f"Wrote {nc_out} after regridding {varname} to regular (latitude, longitude).")

else:
    raise ValueError(
        f"Unsupported shapes:\n"
        f"  {varname} dims: {da.dims}\n"
        f"  {latname} dims: {latv.dims}\n"
        f"  {lonname} dims: {lonv.dims}\n"
        "Expect 1-D lat(row) & lon(col), or 2-D lat(row,col) & lon(row,col)."
    )




Wrote /glade/derecho/scratch/boehnert/AQE/SIP/sept_camxv72_sip_pm25_avg.nc after regridding PM25 to regular (latitude, longitude).


STEP 2 - Convert the netCDF to a Geotiff

In [7]:
import xarray as xr
import rasterio
from rasterio.transform import from_origin
import numpy as np

#CHANGE this based on the netCDF file you want to work with
#
# --- Load NetCDF file ---
netcdf_path = r"/glade/derecho/scratch/boehnert/AQE/SIP/sept_camxv72_sip_pm25_avg.nc"
# --- Save as GeoTIFF ---
tif_path = r"/glade/derecho/scratch/boehnert/AQE/rasters/sept_sip_pm25_2016.tif"

# Open dataset using xarray
ds = xr.open_dataset(netcdf_path)

# --- Inspect available variables ---
print(ds.data_vars)

# Choose your variable — adjust if needed
var_name = "PM25"  # or use print(ds.data_vars) to confirm

# Read variable and coordinates
no2 = ds[var_name][0,0,:,:]
lat = ds['latitude'][:]
lon = ds['longitude'][:]


# Make sure dimensions match
print("NO2 shape:", no2.shape)
print("Lat size:", len(lat))
print("Lon size:", len(lon))

# --- Flip data vertically to match GeoTIFF convention ---
no2_flipped = np.flipud(no2.values)
lat_flipped = lat[::-1]  # To match flipped data

# --- Define transform (top-left corner = NW) ---
res_lon = np.abs(lon[1] - lon[0])
res_lat = np.abs(lat[1] - lat[0])

transform = from_origin(
    west=lon.min() - res_lon / 2,
    north=lat.max() + res_lat / 2,  # Because we flipped lat
    xsize=res_lon,
    ysize=res_lat
)


# --- Handle fill values ---
no2_masked = np.ma.masked_invalid(no2_flipped)
nodata_val = -9999

# --- Save as GeoTIFF ---

print("NO2 masked shape:", no2_masked.shape)
raster_meta = {
    "driver": "GTiff",
    "height": no2_masked.shape[0],
    "width": no2_masked.shape[1],
    "dtype": "float32",
    "count" : 1,
    "crs": "EPSG:4326",
    "transform": transform,
    "nodata": nodata_val
}

with rasterio.open(tif_path, "w", **raster_meta) as dst:
    print("hello")
    dst.write(no2_masked.filled(nodata_val).astype("float32"),1)
   

print(f"✅ Corrected GeoTIFF saved: {tif_path}")


Data variables:
    PM25     (TSTEP, LAY, latitude, longitude) float32 ...
NO2 shape: (142, 183)
Lat size: 142
Lon size: 183
NO2 masked shape: (142, 183)
hello
✅ Corrected GeoTIFF saved: /glade/derecho/scratch/boehnert/AQE/rasters/sept_sip_pm25_2016.tif


STEP 3 - Perform Zonal statistics to aggregate the SIP model to county and tracts

In [8]:
#set up for zonal stats
import geopandas as gpd
from exactextract import exact_extract
import rasterio
import pandas as pd
from shapely.geometry import mapping

#CHANGE this to point to your shapefiles
shapeTract = r"/glade/derecho/scratch/boehnert/AQE/shapefile/Colorado_tracts.shp"
shapeCounty = r"/glade/derecho/scratch/boehnert/AQE/shapefile/Colorado_Counties.shp"

In [9]:
#PM 2.5
#TRACTS aggregation

# Load shapefile and match CRS
tracts = gpd.read_file(shapeTract)
# CHANGE this to point to your geotiff you created in Step 2
#
rast = r"/glade/derecho/scratch/boehnert/AQE/rasters/sept_sip_pm25_2016.tif"
#Output CSV
#
out = r"/glade/derecho/scratch/boehnert/AQE/output/sept_sip_tract_pm25_2016.csv"


# Open NO₂ raster
with rasterio.open(rast) as src:
    # Extract weighted means (GeoJSON-style input)
    #means = [exact_extract(src, mapping(geom), ['mean'])[0]['mean'] for geom in tracts.geometry]
    means = exact_extract(rast, tracts, ["mean"], output='pandas')
    medians = exact_extract(rast, tracts, ["median"], output='pandas')
    std = exact_extract(rast, tracts, ["stdev"], output='pandas')
    count = exact_extract(rast, tracts, ["count"], output='pandas')
    
# Check length
print(len(means))  # Should be 1447


# Assign and export
tracts['MEAN'] = means
tracts['MEDIAN'] = medians
tracts['STD'] = std
tracts["COUNT"] = count


# Export desired fields
tracts[['FIPS', 'SQKM', 'COUNT', 'MEAN', 'MEDIAN', 'STD']].to_csv(out, index=False)

1447


In [5]:
#PM 2.5
#COUNTIES Aggregation

# Load shapefile and match CRS
tracts = gpd.read_file(shapeCounty)
#CHANGE this to point to your geotiff you created in Step 2
#
rast = r"/glade/derecho/scratch/boehnert/AQE/rasters/sip_pm25_2016.tif"
#output CSV
#
out = r"/glade/derecho/scratch/boehnert/AQE/output/sip_country_pm25_2016.csv"

# Open NO₂ raster
with rasterio.open(rast) as src:
    # Extract weighted means (GeoJSON-style input)
    #means = [exact_extract(src, mapping(geom), ['mean'])[0]['mean'] for geom in tracts.geometry]
    means = exact_extract(rast, tracts, ["mean"], output='pandas')
    medians = exact_extract(rast, tracts, ["median"], output='pandas')
    std = exact_extract(rast, tracts, ["stdev"], output='pandas')
    count = exact_extract(rast, tracts, ["count"], output='pandas')
    
# Check length
print(len(means))  # Should be 1447

    
# Assign and export
tracts['MEAN'] = means
tracts['MEDIAN'] = medians
tracts['STD'] = std
tracts["COUNT"] = count


# Export desired fields
tracts[['FIPS', 'SQKM', 'COUNT', 'MEAN', 'MEDIAN', 'STD']].to_csv(out, index=False)


64


Exception: Unhandled feature datatype

DriverError: Failed to read GeoJSON data